<a href="https://colab.research.google.com/github/Ranjana-Kambhammettu/password-generator-and-analyser/blob/main/Password_Generator_and_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collecting Common Passwords and storing them in a csv file

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/")
!ls

MyDrive


In [ ]:
import os
os.chdir("/content/drive/My Drive/Password Generator and Analyzer")
!ls

 common_passwords.csv  'Password Generator and Analyzer.ipynb'
'Leaked Database'


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from nltk.metrics import edit_distance
#import Pycluster as PC
#from pyclustering.cluster.kmedoids import kmedoids

In [ ]:
# Scraping Website 1
url1 = "https://en.wikipedia.org/wiki/Wikipedia:10,000_most_common_passwords"
webdata  = requests.get(url1).text
soup = BeautifulSoup(webdata,"html.parser")
cols = soup.find_all('div',class_='div-col')
data = {"Common Passwords":[]}
for col in cols:
    passwds=col.find_all('li')
    for passwd in passwds:
        if passwd.text.strip()!='':
            data["Common Passwords"].append(passwd.text)
#Scraping Website 2
url2 = 'https://github.com/OWASP/passfault/blob/master/wordlists/wordlists/10k-worst-passwords.txt'
webdata  = requests.get(url2).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_="blob-code blob-code-inner js-file-line")
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)
#Scraping Website 3
url3='https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/10-million-password-list-top-100000.txt'
webdata = requests.get(url3).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_='blob-code blob-code-inner js-file-line')
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)    

#Writing all unique passwrods into a csv file    
common_password = pd.DataFrame.from_dict(data)
common_password = common_password.drop_duplicates(subset='Common Passwords', keep="first")
common_password = common_password.reset_index(drop=True)
common_password = common_password.sample(frac=1)
common_password.to_csv('common_passwords.csv')

In [ ]:
#clustering using AffinityPropagation
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
words=common_password['Common Passwords'].tolist()
words=words[:4500]
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *ruffneck:* jeffbeck, redneck1, ruffneck, ufdyfrecjr
 - *210580:* 110580, 200580, 210180, 210387, 210576, 210580, 210581, 210590, 210592, 210778, 210789, 210980, 210981, 220586, 230680, 250280, 250583, 260580
 - *spawn1:* aspen1, espana, myspace1, snapon1, snowy1, spanky1, sparky1, spartan11, spawn1, spawn2, uconn1
 - *010594:* 010290, 010394, 010574, 010591, 010594, 030564, 050194, 070596, 070894, 090564, 110595, 170594, 310591, 310593
 - *penny:* Kenneth, UpnFMc, decoy, fenway, funnyguy, gegrby, jenni, jennyk, kenned, kenzo, opennow, penny, penny2, phoenix5, porky, wvj5np, yesno, zenit
 - *angina:* Angelina, Baggins, alaina, albino, angina, anointed, antonino, asenna, badgirls, cincinna, edwina, gina, landing, sanguine, yugioh
 - *ofborg:* kiborg, ndbyrb, nfhfrfy, ofborg, uhfdbwfgf, yCWVrxXH
 - *10061958:* 100198, 10061957, 10061958, 10061985, 10061994, 10081988, 11061968, 14071958, 15041954, 17061954, 17061956, 19061988, 20071959, 30081952, 30091988
 - *angel3:* ace2luv, anabelle

Making Trie Class

In [ ]:
class TrieNode:
    def __init__(self):
        self.children={}
        self.endOfWord = False
class Trie:
    def __init__(self):
        self.root = TrieNode()
    def insert(self,word):
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur=cur.children[c]
        cur.endOfWord = True
    def search(self,word):
        cur = self.root
        for c in word:
            if c not in cur.children:
                return False
            cur = cur.children[c]
        return cur.endOfWord

In [ ]:
breached_passwds=Trie()
import os
l=os.getcwd()+"/Leaked Database"
a=os.listdir(l)
for i in a:
    p=l+"/"+i
    file = open(p, 'r',errors='ignore')
    for j in file:
        breached_passwds.insert(j)



In [ ]:
#Implementing Algorithm
import random
words = input("Enter five words(give space between each word): ").split()
numbers = input("Enter two numbers(give space between each number): ").split()
num_passwords = int(input("Enter number of passwords required:"))
special_chars = ['@','$','!','#','%','&','(',')','0','3','8','<','|']
punc = ['@','*','+','-',':','”','/','\\','~','?','[',']','{','}','$','!','#','%','&','(',')','_' ,'<','|']
alphabets = ['a','s','i','r','x','q','c','j','o','e','b','k','l']
c2s = {}
for i in range(13):
  c2s[alphabets[i]]=special_chars[i]
def randomly_capitalize(s):
  result = ''
  for char in s:
      flag = random.randint(0, 1)
      if flag == 0:
          result += char.upper()
      else:
          result += char.lower()
  return result
def merge_randomly(str1,str2):
  result = ''
  flag = random.randint(0, 1)
  if flag == 0:
      result = str1 + str2
  else:
      result = str2 + str1
  return result,flag
def charToSpecial(s):
  for char in s:
    if char in c2s:
      s = s.replace(char,c2s[char])
  return s
def insert_number(s,n,f,str1,str2):
  flag = random.randint(0, 2)
  if flag == 0:
    return n+s
  elif flag == 2:
    return s+n
  else:
    if f==0:
      return s[:len(str1)]+n+s[len(str1):]
    else:
      return s[:len(str2)]+n+s[len(str2):]
def add_punctuation(s):
  return s+random.choice(punc)
while num_passwords>0:
  temp = list(words)
  string1 = random.choice(temp)
  temp.remove(string1)
  string2 = random.choice(temp)
  number = random.choice(numbers)
  string1,string2 = randomly_capitalize(string1),randomly_capitalize(string2)
  final_string,f = merge_randomly(string1,string2)
  final_string = charToSpecial(final_string)
  final_string = insert_number(final_string,number,f,string1,string2)
  final_string = add_punctuation(final_string)
  C = len(final_string)
  if C < 8:
    for i in range(8 - C):
      final_string += random.choice(special_chars)
  print("Final Passowrd:",final_string)
  num_passwords -= 1
  

Enter five words(give space between each word): a b c d e
Enter two numbers(give space between each number): 2 3
Enter number of passwords required:3
Final Passowrd: 2d8<!(<)
Final Passowrd: BC2-80!!
Final Passowrd: (32]#<%<
